# Grouping for Aggregation, Filtration, and Transformation

In [5]:
import os
import pandas as pd
import numpy as np

os.chdir("../../coding_data/cookbook_data/")
os.getcwd()

"C:\\Users\\k2035549\\OneDrive - King's College London\\Desktop\\Coding\\coding_data\\cookbook_data"

In [6]:
import pandas as pd
import numpy as np
# pd.set_option('max_columns', 4, 'max_rows', 10, 'max_colwidth', 12)
pd.set_option('display.max_columns', None)

# Content Page

With any kind of grouping operation, it helps to identify the three components:
* grouping columns
* aggregating columns
* aggregating functions

groupby object has four methods that accept a function to perform a calculation on each group.
* .agg (must retrun scalar)
* .filter (return a boolean)
* .transform (return series or df with the same length as the passed group)
* .apply (return scalar, series or df)


1. Defining an Aggregation - Showing the syntaxes for pandas to groupby
2. Grouping and aggregating with multiple columns and functions
    * (Using named aggregation object that can create non-hierarchical columns)
3. Removing the MultiIndex after grouping
4. Grouping with a custom aggregation function
5. Customizing aggregating functions with *args and **kwargs
6. Filtering groups in groupby object (Filtering for states with a minority majority)
    * Index of dataframe after filtering is index of original df and not groupby columns
7. Transforming through a weight loss bet
8. Calculating weighted mean SAT scores per state with apply
9. Grouping by continuous variables
10. Counting the total number of flights between cities
11. Finding the longest streak of on-time flights

# Load data

In [11]:
college = pd.read_csv('data/college.csv')
flights = pd.read_csv('data/flights.csv')

## Introduction

### Defining an Aggregation

Showing the syntaxes for pandas to groupby

### How to do it...

In [4]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


Different ways to aggregate in pandas

In [5]:
(flights
     .groupby('AIRLINE')
     .agg({'ARR_DELAY':'mean'})
)

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


In [6]:
(flights
     .groupby('AIRLINE')
     ['ARR_DELAY']
     .agg('mean')
)

AIRLINE
AA     5.542661
AS    -0.833333
B6     8.692593
DL     0.339691
EV     7.034580
F9    13.630651
HA     4.972973
MQ     6.860591
NK    18.436070
OO     7.593463
UA     7.765755
US     1.681105
VX     5.348884
WN     6.397353
Name: ARR_DELAY, dtype: float64

In [7]:
(flights
    .groupby('AIRLINE')
    ['ARR_DELAY']
    .agg(np.mean)
)

AIRLINE
AA     5.542661
AS    -0.833333
B6     8.692593
DL     0.339691
EV     7.034580
F9    13.630651
HA     4.972973
MQ     6.860591
NK    18.436070
OO     7.593463
UA     7.765755
US     1.681105
VX     5.348884
WN     6.397353
Name: ARR_DELAY, dtype: float64

In [8]:
(flights
    .groupby('AIRLINE')
    ['ARR_DELAY']
    .mean()
)

AIRLINE
AA     5.542661
AS    -0.833333
B6     8.692593
DL     0.339691
EV     7.034580
F9    13.630651
HA     4.972973
MQ     6.860591
NK    18.436070
OO     7.593463
UA     7.765755
US     1.681105
VX     5.348884
WN     6.397353
Name: ARR_DELAY, dtype: float64

### How it works...

In [9]:
grouped = flights.groupby('AIRLINE')
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

### There's more...

In [8]:
(flights
   .groupby('AIRLINE')
   ['ARR_DELAY']
   .agg(np.sqrt)
)

/Users/matt/.env/364/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Exception: Must produce aggregated value

## Grouping and aggregating with multiple columns and functions
Also:
Using named aggregation object that can create non-hierarchical columns

### How to do it...

Find the number of canceled flights for every airline per weekday

In [25]:
# output series
(flights
    .groupby(['AIRLINE', 'WEEKDAY'])
    ['CANCELLED'] 
    .agg('sum')
)

# # output df
# (flights
#     .groupby(['AIRLINE', 'WEEKDAY'])
#     .agg({'CANCELLED':'mean'})
# )


AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
                    ..
WN       3          18
         4          10
         5           7
         6          10
         7           7
Name: CANCELLED, Length: 98, dtype: int64

Finding the number and percentage of canceled and diverted flights for every airline per weekday

In [32]:
(flights
    .groupby(['AIRLINE', 'WEEKDAY']) 
    ['CANCELLED', 'DIVERTED']
    .agg(['sum', 'mean'])
)

# # when using dictionary as inout for .agg
# (flights
#     .groupby(['AIRLINE', 'WEEKDAY'])
#     .agg({'CANCELLED': ['sum', 'mean'],
#           'DIVERTED': ['sum', 'mean']})
# )

<ipython-input-32-c391237825a8>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  (flights


CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
...                   ...       ...      ...       ...
WN      3              18  0.014118        2  0.001569
        4              10  0.007911        4  0.003165
        5               7  0.005828        0  0.000000
        6              10  0.010132        3  0.003040
        7               7  0.006066        3  0.002600

[98 rows x 4 columns]

For each origin and destination, finding the total number of flights, the number and percentage of canceled flights and the average and variance of the airtime

In [13]:
(flights
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
)

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

Using named aggregation object that can create non-hierarchical columns

In [35]:
(flights
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg(sum_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='sum'),
         mean_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='mean'),
         size_cancelled=pd.NamedAgg(column='CANCELLED', aggfunc='size'),
         mean_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='mean'),
         var_air_time=pd.NamedAgg(column='AIR_TIME', aggfunc='var'))
)

sum_cancelled  mean_cancelled  size_cancelled  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0        0.000000              31   
        ABQ                   0        0.000000              16   
        ABY                   0        0.000000              19   
        ACY                   0        0.000000               6   
        AEX                   0        0.000000              40   
...                         ...             ...             ...   
SFO     SNA                   4        0.032787             122   
        STL                   0        0.000000              20   
        SUN                   0        0.000000              10   
        TUS                   0        0.000000              20   
        XNA                   0        0.000000               2   

                  mean_air_time  var_air_time  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692  
...                         ...           ...  
SFO     SNA           64.059322     11.338331  
        STL          198.900000    101.042105  
        SUN           78.000000     25.777778  
        TUS          100.200000     35.221053  
        XNA          173.500000      0.500000  

[1130 rows x 5 columns]

### How it works...

### There's more...

Flattening column names (specifically for third step above)

In [36]:
res = (flights
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
)
res.columns = ['_'.join(x) for x in
    res.columns.to_flat_index()]

In [37]:
res

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0        0.000000              31   
        ABQ                   0        0.000000              16   
        ABY                   0        0.000000              19   
        ACY                   0        0.000000               6   
        AEX                   0        0.000000              40   
...                         ...             ...             ...   
SFO     SNA                   4        0.032787             122   
        STL                   0        0.000000              20   
        SUN                   0        0.000000              10   
        TUS                   0        0.000000              20   
        XNA                   0        0.000000               2   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692  
...                         ...           ...  
SFO     SNA           64.059322     11.338331  
        STL          198.900000    101.042105  
        SUN           78.000000     25.777778  
        TUS          100.200000     35.221053  
        XNA          173.500000      0.500000  

[1130 rows x 5 columns]

In [38]:
def flatten_cols(df):
    df.columns = ['_'.join(x) for x in
        df.columns.to_flat_index()]
    return df

In [39]:
res = (flights
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
    .pipe(flatten_cols)
)

Hierarchical index with categorical columns. res has 1,130 rows but when ORG_AIR has become categorical column, 2710 rows form! This is because pandas will create a cartesian product of all combinations for each level.

In [40]:
res

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0        0.000000              31   
        ABQ                   0        0.000000              16   
        ABY                   0        0.000000              19   
        ACY                   0        0.000000               6   
        AEX                   0        0.000000              40   
...                         ...             ...             ...   
SFO     SNA                   4        0.032787             122   
        STL                   0        0.000000              20   
        SUN                   0        0.000000              10   
        TUS                   0        0.000000              20   
        XNA                   0        0.000000               2   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692  
...                         ...           ...  
SFO     SNA           64.059322     11.338331  
        STL          198.900000    101.042105  
        SUN           78.000000     25.777778  
        TUS          100.200000     35.221053  
        XNA          173.500000      0.500000  

[1130 rows x 5 columns]

In [41]:
res = (flights
    .assign(ORG_AIR=flights.ORG_AIR.astype('category'))
    .groupby(['ORG_AIR', 'DEST_AIR'])
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
)
res

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE            0.0  0.0   31   96.387097  45.778495
        ABI            NaN  NaN    0         NaN        NaN
        ABQ            0.0  0.0   16  170.500000  87.866667
        ABR            NaN  NaN    0         NaN        NaN
        ABY            0.0  0.0   19   28.578947   6.590643
...                    ...  ...  ...         ...        ...
SFO     TYS            NaN  NaN    0         NaN        NaN
        VLD            NaN  NaN    0         NaN        NaN
        VPS            NaN  NaN    0         NaN        NaN
        XNA            0.0  0.0    2  173.500000   0.500000
        YUM            NaN  NaN    0         NaN        NaN

[2710 rows x 5 columns]

In [42]:
res = (flights
    .assign(ORG_AIR=flights.ORG_AIR.astype('category'))
    .groupby(['ORG_AIR', 'DEST_AIR'], observed=True)
    .agg({'CANCELLED':['sum', 'mean', 'size'],
          'AIR_TIME':['mean', 'var']})
)
res

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
LAX     ABQ              1  0.018182   55   89.259259   29.403215
        ANC              0  0.000000    7  307.428571   78.952381
        ASE              1  0.038462   26  102.920000  102.243333
        ATL              0  0.000000  174  224.201149  127.155837
        AUS              0  0.000000   80  150.537500   57.897310
...                    ...       ...  ...         ...         ...
MSP     TTN              1  0.125000    8  124.428571   57.952381
        TUL              0  0.000000   18   91.611111   63.075163
        TUS              0  0.000000    2  176.000000   32.000000
        TVC              0  0.000000    5   56.600000   10.300000
        XNA              0  0.000000   14   90.642857  115.939560

[1130 rows x 5 columns]

## Removing the MultiIndex after grouping

Find the total and average miles flown and the maximum and minimum arrival delay for each airline for each weekday

In [56]:
flights = pd.read_csv('data/flights.csv')
airline_info = (flights
    .groupby(['AIRLINE', 'WEEKDAY'])
    .agg({'DIST':['sum', 'mean'],
          'ARR_DELAY':['min', 'max']}) 
    .astype(int)
)
airline_info

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
...                  ...   ...       ...  ...
WN      3         997213   782       -38  262
        4        1024854   810       -52  284
        5         981036   816       -44  244
        6         823946   834       -41  290
        7         945679   819       -45  261

[98 rows x 4 columns]

 Look at the multi levels for column and index names

In [47]:
airline_info.columns.get_level_values(0)

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [48]:
airline_info.columns.get_level_values(1)

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [49]:
airline_info.columns.to_flat_index()

Index([('DIST', 'sum'), ('DIST', 'mean'), ('ARR_DELAY', 'min'),
       ('ARR_DELAY', 'max')],
      dtype='object')

In [50]:
airline_info.columns = ['_'.join(x) for x in
    airline_info.columns.to_flat_index()]

In [51]:
airline_info

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732
...                   ...        ...            ...            ...
WN      3          997213        782            -38            262
        4         1024854        810            -52            284
        5          981036        816            -44            244
        6          823946        834            -41            290
        7          945679        819            -45            261

[98 rows x 4 columns]

In [52]:
# quick way to get rid of multiindex
airline_info.reset_index()

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
...,...,...,...,...,...,...
93,WN,3,997213,782,-38,262
94,WN,4,1024854,810,-52,284
95,WN,5,981036,816,-44,244
96,WN,6,823946,834,-41,290


In [53]:
(flights
    .groupby(['AIRLINE', 'WEEKDAY'])
    .agg(dist_sum=pd.NamedAgg(column='DIST', aggfunc='sum'),
         dist_mean=pd.NamedAgg(column='DIST', aggfunc='mean'),
         arr_delay_min=pd.NamedAgg(column='ARR_DELAY', aggfunc='min'),
         arr_delay_max=pd.NamedAgg(column='ARR_DELAY', aggfunc='max'))
    .astype(int)
    .reset_index()
)

,AIRLINE,WEEKDAY,dist_sum,dist_mean,arr_delay_min,arr_delay_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
...,...,...,...,...,...,...
93,WN,3,997213,782,-38,262
94,WN,4,1024854,810,-52,284
95,WN,5,981036,816,-44,244
96,WN,6,823946,834,-41,290


### How it works...

### There's more...

In [58]:
(flights
    .groupby(['AIRLINE'], as_index=False)
    ['DIST']
    .agg('mean')
    .round(0)
)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0
7,MQ,404.0
8,NK,1047.0
9,OO,511.0


## Grouping with a custom aggregation function

### How to do it...

Find mean and std of the undergrad population per state
Then find the maximum number of standard deviations from the mean that any single population value is per state

In [60]:
college = pd.read_csv('data/college.csv')
(college
    .groupby('STABBR')
    ['UGDS']
    .agg(['mean', 'std'])
    .round(0)
).head(3)

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0


Then find the maximum number of standard deviations from the mean that any single population value is per state

In [61]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [62]:
# NB: NaNs can be raised as in above we calculate the std but some groups might have only one value and so error occurs
(college
    .groupby('STABBR')
    ['UGDS']
    .agg(max_deviation)
    .round(1)
)

STABBR
AK     2.6
AL     5.8
AR     6.3
AS     NaN
AZ     9.9
CA     6.1
CO     5.0
CT     5.6
DC     2.4
DE     3.5
FL     8.4
FM     NaN
GA     5.4
GU     1.0
HI     3.8
IA     6.5
ID     4.5
IL     7.3
IN     9.1
KS     4.9
KY     5.2
LA     6.5
MA     6.1
MD     5.3
ME     4.0
MH     NaN
MI     6.7
MN     7.8
MO     7.2
MP     NaN
MS     4.0
MT     3.9
NC     4.9
ND     3.5
NE     5.0
NH     5.3
NJ     7.1
NM     4.5
NV     4.7
NY     8.2
OH    10.3
OK     5.9
OR     5.3
PA    10.1
PR     6.0
PW     NaN
RI     2.9
SC     6.0
SD     4.2
TN     6.0
TX     7.7
UT     5.1
VA     7.0
VI     NaN
VT     3.8
WA     6.6
WI     5.8
WV     7.2
WY     2.8
Name: UGDS, dtype: float64

### How it works...

### There's more...

In [32]:
# apply custom func to multiple columns
(college
    .groupby('STABBR')
    ['UGDS', 'SATVRMID', 'SATMTMID']
    .agg(max_deviation)
    .round(1)
)

,UGDS,SATVRMID,SATMTMID
STABBR,,,
AK,2.6,NaN,NaN
AL,5.8,1.6,1.8
AR,6.3,2.2,2.3
AS,NaN,NaN,NaN
AZ,9.9,1.9,1.4
...,...,...,...
VT,3.8,1.9,1.9
WA,6.6,2.2,2.0
WI,5.8,2.4,2.2


In [33]:
# use custom and standard agg functions
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATVRMID', 'SATMTMID'] 
    .agg([max_deviation, 'mean', 'std'])
    .round(1)
)

UGDS          ... SATMTMID      
                max_deviation    mean  ...     mean   std
STABBR RELAFFIL                        ...               
AK     0                 2.1   3508.9  ...      NaN   NaN
       1                 1.1    123.3  ...    503.0   NaN
AL     0                 5.2   3248.8  ...    515.8  56.7
       1                 2.4    979.7  ...    485.6  61.4
AR     0                 5.8   1793.7  ...    503.6  39.0
...                      ...      ...  ...      ...   ...
WI     0                 5.3   2879.1  ...    591.2  85.7
       1                 3.4   1716.2  ...    526.6  42.5
WV     0                 6.9   1873.9  ...    480.0  27.7
       1                 1.3    716.4  ...    484.8  17.7
WY     0                 2.8   2244.4  ...    540.0   NaN

[112 rows x 9 columns]

In [34]:
max_deviation.__name__

'max_deviation'

In [35]:
max_deviation.__name__ = 'Max Deviation'
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATVRMID', 'SATMTMID'] 
    .agg([max_deviation, 'mean', 'std'])
    .round(1)
)

UGDS          ... SATMTMID      
                Max Deviation    mean  ...     mean   std
STABBR RELAFFIL                        ...               
AK     0                 2.1   3508.9  ...      NaN   NaN
       1                 1.1    123.3  ...    503.0   NaN
AL     0                 5.2   3248.8  ...    515.8  56.7
       1                 2.4    979.7  ...    485.6  61.4
AR     0                 5.8   1793.7  ...    503.6  39.0
...                      ...      ...  ...      ...   ...
WI     0                 5.3   2879.1  ...    591.2  85.7
       1                 3.4   1716.2  ...    526.6  42.5
WV     0                 6.9   1873.9  ...    480.0  27.7
       1                 1.3    716.4  ...    484.8  17.7
WY     0                 2.8   2244.4  ...    540.0   NaN

[112 rows x 9 columns]

## Customizing aggregating functions with *args and **kwargs

### How to do it...

In [12]:
def pct_between_1_3k(s):
    return (s
        .between(1_000, 3_000)
        .mean()
        * 100
    )

In [13]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] # undergraduate enrollment
    .agg(pct_between_1_3k)
    .round(1)
)

STABBR  RELAFFIL
AK      0           14.3
        1            0.0
AL      0           23.6
        1           33.3
AR      0           27.9
                    ... 
WI      0           13.8
        1           36.0
WV      0           24.6
        1           37.5
WY      0           54.5
Name: UGDS, Length: 112, dtype: float64

In [14]:
def pct_between(s, low, high):
    return s.between(low, high).mean() * 100

In [15]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg(pct_between, 1_000, 10_000)
    .round(1)
)

STABBR  RELAFFIL
AK      0           42.9
        1            0.0
AL      0           45.8
        1           37.5
AR      0           39.7
                    ... 
WI      0           31.0
        1           44.0
WV      0           29.2
        1           37.5
WY      0           72.7
Name: UGDS, Length: 112, dtype: float64

### How it works...

### There's more...

In [16]:
def between_n_m(n, m):
    def wrapper(ser):
        return pct_between(ser, n, m)
    wrapper.__name__ = f'between_{n}_{m}'
    return wrapper

In [17]:
(college
    .groupby(['STABBR', 'RELAFFIL'])
    ['UGDS'] 
    .agg([between_n_m(1_000, 10_000), 'max', 'mean'])
    .round(1)
)

between_1000_10000      max    mean
STABBR RELAFFIL                                     
AK     0                       42.9  12865.0  3508.9
       1                        0.0    275.0   123.3
AL     0                       45.8  29851.0  3248.8
       1                       37.5   3033.0   979.7
AR     0                       39.7  21405.0  1793.7
...                             ...      ...     ...
WI     0                       31.0  29302.0  2879.1
       1                       44.0   8212.0  1716.2
WV     0                       29.2  44924.0  1873.9
       1                       37.5   1375.0   716.4
WY     0                       72.7   9910.0  2244.4

[112 rows x 3 columns]

## Examining the groupby object

### How to do it...

In [28]:
college = pd.read_csv('data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [19]:
print([attr for attr in dir(grouped) if not
    attr.startswith('_')])

['CITY', 'CURROPER', 'DISTANCEONLY', 'GRAD_DEBT_MDN_SUPP', 'HBCU', 'INSTNM', 'MD_EARN_WNE_P10', 'MENONLY', 'PCTFLOAN', 'PCTPELL', 'PPTUG_EF', 'RELAFFIL', 'SATMTMID', 'SATVRMID', 'STABBR', 'UG25ABV', 'UGDS', 'UGDS_2MOR', 'UGDS_AIAN', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_NHPI', 'UGDS_NRA', 'UGDS_UNKN', 'UGDS_WHITE', 'WOMENONLY', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


In [20]:
grouped.ngroups

112

In [21]:
groups = list(grouped.groups)
groups[:6]

[('AK', 0), ('AK', 1), ('AL', 0), ('AL', 1), ('AR', 0), ('AR', 1)]

In [22]:
grouped.get_group(('FL', 1))

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
712,The Baptist College of Florida,Graceville,FL,0.0,0.0,0.0,1,545.0,465.0,0.0,454.0,0.7996,0.0639,0.0441,0.0044,0.0066,0.0000,0.0308,0.0000,0.0507,0.2291,1,0.5878,0.5602,0.3531,30800,20052
713,Barry University,Miami,FL,0.0,0.0,0.0,1,470.0,462.0,0.0,3913.0,0.2004,0.3177,0.2867,0.0153,0.0038,0.0015,0.0164,0.0741,0.0841,0.1518,1,0.5045,0.6733,0.4361,44100,28250
714,Gooding Institute of Nurse Anesthesia,Panama City,FL,0.0,0.0,0.0,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,PrivacySuppressed
715,Bethune-Cookman University,Daytona Beach,FL,1.0,0.0,0.0,1,405.0,395.0,0.0,3897.0,0.0190,0.8891,0.0277,0.0021,0.0010,0.0018,0.0198,0.0205,0.0190,0.0523,1,0.7758,0.8867,0.0647,29400,36250
724,Johnson University Florida,Kissimmee,FL,0.0,0.0,0.0,1,480.0,470.0,0.0,220.0,0.7000,0.1136,0.1545,0.0091,0.0000,0.0000,0.0045,0.0045,0.0136,0.1636,1,0.6689,0.7384,0.2185,26300,20199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7486,Strayer University-Coral Springs Campus,Coral Springs,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,49200,36173.5
7487,Strayer University-Fort Lauderdale Campus,Fort Lauderdale,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,49200,36173.5
7488,Strayer University-Miramar Campus,Miramar,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,49200,36173.5
7489,Strayer University-Doral,Miami,FL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,49200,36173.5


In [30]:
from IPython.display import display
for name, group in grouped:
    print(name)
    display(group.head(3))

In [31]:
for name, group in grouped:
    print(name)
    print(group)
    break

('AK', 0)
                                      INSTNM       CITY STABBR  HBCU  MENONLY  \
60            University of Alaska Anchorage  Anchorage     AK   0.0      0.0   
62            University of Alaska Fairbanks  Fairbanks     AK   0.0      0.0   
63            University of Alaska Southeast     Juneau     AK   0.0      0.0   
65    AVTEC-Alaska's Institute of Technology     Seward     AK   0.0      0.0   
66                 Charter College-Anchorage  Anchorage     AK   0.0      0.0   
67                     Alaska Career College  Anchorage     AK   0.0      0.0   
5171                       Ilisagvik College     Barrow     AK   0.0      0.0   

      WOMENONLY  RELAFFIL  SATVRMID  SATMTMID  DISTANCEONLY     UGDS  \
60          0.0         0       NaN       NaN           0.0  12865.0   
62          0.0         0       NaN       NaN           0.0   5536.0   
63          0.0         0       NaN       NaN           0.0   1428.0   
65          0.0         0       NaN       NaN        

In [25]:
grouped.head(2)

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
10,Birmingham Southern College,Birmingham,AL,0.0,0.0,0.0,1,560.0,560.0,0.0,1180.0,0.7983,0.1102,0.0195,0.0517,0.0102,0.0000,0.0051,0.0000,0.0051,0.0017,1,0.1920,0.4809,0.0152,44200,27000
43,Prince Institute-Southeast,Elmhurst,IL,0.0,0.0,0.0,0,NaN,NaN,0.0,84.0,0.7976,0.1310,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0.7857,0.9375,0.6569,PrivacySuppressed,20992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Pacific Islands University,Mangilao,GU,0.0,0.0,0.0,1,NaN,NaN,0.0,65.0,0.0154,0.0000,0.0000,0.0308,0.0000,0.9538,0.0000,0.0000,0.0000,0.1846,1,0.9730,0.0000,0.2533,PrivacySuppressed,PrivacySuppressed
6439,Touro University Nevada,Henderson,NV,0.0,0.0,0.0,1,NaN,NaN,0.0,31.0,0.5161,0.0000,0.0000,0.3548,0.0000,0.0323,0.0323,0.0000,0.0645,0.0323,1,0.0000,0.2000,0.4000,NaN,PrivacySuppressed
7352,Marinello School of Beauty-Henderson,Henderson,NV,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,21200,9796.5
7404,University of the Virgin Islands-Albert A. Sheen,St. Croix,VI,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,31800,15150


### How it works...

### There's more...

In [26]:
grouped.nth([1, -1])

INSTNM  \
STABBR RELAFFIL                                               
AK     0                     University of Alaska Fairbanks   
       0                                  Ilisagvik College   
       1                          Alaska Pacific University   
       1                           Alaska Christian College   
AL     0                University of Alabama at Birmingham   
...                                                     ...   
WV     0         BridgeValley Community & Technical College   
       1                          Appalachian Bible College   
       1         West Virginia Business College-Nutter Fort   
WY     0                            Central Wyoming College   
       0                            CollegeAmerica-Cheyenne   

                              CITY  HBCU  MENONLY  WOMENONLY  SATVRMID  \
STABBR RELAFFIL                                                          
AK     0                 Fairbanks   0.0      0.0        0.0       NaN   
       0                    Barrow   0.0      0.0        0.0       NaN   
       1                 Anchorage   0.0      0.0        0.0     555.0   
       1                  Soldotna   0.0      0.0        0.0       NaN   
AL     0                Birmingham   0.0      0.0        0.0     570.0   
...                            ...   ...      ...        ...       ...   
WV     0         South  Charleston   0.0      0.0        0.0       NaN   
       1                Mount Hope   0.0      0.0        0.0     514.0   
       1               Nutter Fort   NaN      NaN        NaN       NaN   
WY     0                  Riverton   0.0      0.0        0.0       NaN   
       0                  Cheyenne   0.0      0.0        0.0       NaN   

                 SATMTMID  DISTANCEONLY     UGDS  UGDS_WHITE  UGDS_BLACK  \
STABBR RELAFFIL                                                            
AK     0              NaN           0.0   5536.0      0.4259      0.0210   
       0              NaN           0.0    109.0      0.1376      0.0183   
       1            503.0           0.0    275.0      0.5309      0.0291   
       1              NaN           0.0     68.0      0.0588      0.0000   
AL     0            565.0           0.0  11383.0      0.5922      0.2600   
...                   ...           ...      ...         ...         ...   
WV     0              NaN           0.0   1887.0      0.8967      0.0753   
       1            487.0           0.0    196.0      0.9643      0.0000   
       1              NaN           NaN      NaN         NaN         NaN   
WY     0              NaN           0.0   1099.0      0.7097      0.0136   
       0              NaN           0.0     60.0      0.6500      0.0667   

                 UGDS_HISP  UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  UGDS_2MOR  \
STABBR RELAFFIL                                                           
AK     0            0.0522      0.0126     0.1284     0.0027     0.0401   
       0            0.0092      0.0826     0.6881     0.0459     0.0000   
       1            0.0364      0.0255     0.1855     0.0109     0.0945   
       1            0.0147      0.0000     0.7794     0.0000     0.0147   
AL     0            0.0283      0.0518     0.0022     0.0007     0.0368   
...                    ...         ...        ...        ...        ...   
WV     0            0.0042      0.0069     0.0154     0.0000     0.0000   
       1            0.0153      0.0000     0.0051     0.0102     0.0000   
       1               NaN         NaN        NaN        NaN        NaN   
WY     0            0.0846      0.0055     0.1301     0.0018     0.0300   
       0            0.2000      0.0000     0.0000     0.0000     0.0000   

                 UGDS_NRA  UGDS_UNKN  PPTUG_EF  CURROPER  PCTPELL  PCTFLOAN  \
STABBR RELAFFIL                                                               
AK     0           0.0110     0.3060    0.3887         1   0.2263    0.2550   
       0           0.0183     0.0000    0.6239         1   0.1323   

## Filtering for states with a minority majority


NB: .filter function for groupby method acts different to usual, you input a function to outputs True or False per group to see if we keep it

### How to do it...

In [32]:
college = pd.read_csv('data/college.csv', index_col='INSTNM')
grouped = college.groupby('STABBR')
grouped.ngroups

59

In [33]:
college['STABBR'].nunique() # verifying the same number # State abbreviation

59

In [34]:
def check_minority(df, threshold):
    minority_pct = 1 - df['UGDS_WHITE']
    total_minority = (df['UGDS'] * minority_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_minority_pct = total_minority / total_ugds
    return total_minority_pct > threshold

In [35]:
college_filtered = grouped.filter(check_minority, threshold=.5)
college_filtered

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,0.3162,0.4405,0.0763,0.0017,0.0207,0.0046,0.0373,0.0,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,0.3253,0.0843,0.1566,0.0000,0.0241,0.0000,0.0241,0.0,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,0.7600,0.0400,0.1200,0.0000,0.0000,0.0400,0.0400,0.0,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,0.2143,0.0873,0.5794,0.0159,0.0873,0.0079,0.0000,0.0,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,1.0,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacySuppressed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WestMed College - Merced,Merced,CA,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,15623.5
Vantage College,El Paso,TX,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,9500
SAE Institute of Technology San Francisco,Emeryville,CA,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,9500


In [36]:
college.shape

(7535, 26)

In [37]:
college_filtered.shape

(3028, 26)

In [38]:
college_filtered['STABBR'].nunique()

20

### How it works...

### There's more...

In [39]:
college_filtered_20 = grouped.filter(check_minority, threshold=.2)
college_filtered_20.shape

(7461, 26)

In [40]:
college_filtered_20['STABBR'].nunique()

57

In [41]:
college_filtered_70 = grouped.filter(check_minority, threshold=.7)
college_filtered_70.shape

(957, 26)

In [42]:
college_filtered_70['STABBR'].nunique()

10

## Transforming through a weight loss bet

.transform returns object that preserves index of the orginal Dataframe but allows you to do calculations on groups of the data
.transform useful for summarizing infromation from the groups and then adding it back to the orginal dataframe

### How to do it...

In [43]:
weight_loss = pd.read_csv('data/weight_loss.csv')
weight_loss.query('Month == "Jan"')

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190


In [44]:
def percent_loss(s):
    return ((s - s.iloc[0]) / s.iloc[0]) * 100

In [45]:
(weight_loss
    .query('Name=="Bob" and Month=="Jan"')
    ['Weight']
    .pipe(percent_loss)
)

0    0.000000
2   -1.030928
4   -2.749141
6   -2.749141
Name: Weight, dtype: float64

In [46]:
(weight_loss
    .groupby(['Name', 'Month'])
    ['Weight'] 
    .transform(percent_loss)
)

0     0.000000
1     0.000000
2    -1.030928
3    -4.060914
4    -2.749141
5    -4.060914
6    -2.749141
7    -3.553299
8     0.000000
9     0.000000
10   -2.826855
11   -3.157895
12   -5.300353
13   -6.842105
14   -5.300353
15   -8.947368
16    0.000000
17    0.000000
18    1.119403
19    0.000000
20   -1.119403
21   -1.734104
22   -2.611940
23   -1.734104
24    0.000000
25    0.000000
26   -1.149425
27   -3.529412
28   -3.065134
29   -3.529412
30   -4.214559
31   -5.294118
Name: Weight, dtype: float64

In [47]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Name=="Bob" and Month in ["Jan", "Feb"]')
)

,Name,Month,Week,Weight,percent_loss
0,Bob,Jan,Week 1,291,0.0
2,Bob,Jan,Week 2,288,-1.0
4,Bob,Jan,Week 3,283,-2.7
6,Bob,Jan,Week 4,283,-2.7
8,Bob,Feb,Week 1,283,0.0
10,Bob,Feb,Week 2,275,-2.8
12,Bob,Feb,Week 3,268,-5.3
14,Bob,Feb,Week 4,268,-5.3


In [48]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
)

,Name,Month,Week,Weight,percent_loss
6,Bob,Jan,Week 4,283,-2.7
7,Amy,Jan,Week 4,190,-3.6
14,Bob,Feb,Week 4,268,-5.3
15,Amy,Feb,Week 4,173,-8.9
22,Bob,Mar,Week 4,261,-2.6
23,Amy,Mar,Week 4,170,-1.7
30,Bob,Apr,Week 4,250,-4.2
31,Amy,Apr,Week 4,161,-5.3


In [49]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
)

Name,Amy,Bob
Month,,
Apr,-5.3,-4.2
Feb,-8.9,-5.3
Jan,-3.6,-2.7
Mar,-1.7,-2.6


In [50]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
)

Name,Amy,Bob,winner
Month,,,
Apr,-5.3,-4.2,Amy
Feb,-8.9,-5.3,Amy
Jan,-3.6,-2.7,Amy
Mar,-1.7,-2.6,Bob


In [51]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
    .style.highlight_min(axis=1)
)

Name,Amy,Bob,winner
Month,,,
Apr,-5.300000,-4.200000,Amy
Feb,-8.900000,-5.300000,Amy
Jan,-3.600000,-2.700000,Amy
Mar,-1.700000,-2.600000,Bob


In [52]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
    .assign(winner=lambda df_:
            np.where(df_.Amy < df_.Bob, 'Amy', 'Bob'))
    .winner
    .value_counts()
)

Amy    3
Bob    1
Name: winner, dtype: int64

### How it works...

In [53]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)))
    .query('Week == "Week 4"')
    .groupby(['Month', 'Name'])
    ['percent_loss']
    .first()
    .unstack()
)

Name,Amy,Bob
Month,,
Apr,-5.3,-4.2
Feb,-8.9,-5.3
Jan,-3.6,-2.7
Mar,-1.7,-2.6


### There's more...

In [54]:
(weight_loss
    .assign(percent_loss=(weight_loss
        .groupby(['Name', 'Month'])
        ['Weight'] 
        .transform(percent_loss)
        .round(1)),
            Month=pd.Categorical(weight_loss.Month,
                  categories=['Jan', 'Feb', 'Mar', 'Apr'],
                  ordered=True))
    .query('Week == "Week 4"')
    .pivot(index='Month', columns='Name',
           values='percent_loss')
)

Name,Amy,Bob
Month,,
Jan,-3.6,-2.7
Feb,-8.9,-5.3
Mar,-1.7,-2.6
Apr,-5.3,-4.2


## Calculating weighted mean SAT scores per state with apply

For math and verbal SAT scores

Weight the scores by population of undergraduate students per school.

### How to do it...

In [55]:
college = pd.read_csv('data/college.csv')
subset = ['UGDS', 'SATMTMID', 'SATVRMID'] # undergrad enrollment, SAT math median, SAT verbal median
college2 = college.dropna(subset=subset)
college.shape

(7535, 27)

In [56]:
college2.shape

(1184, 27)

In [57]:
def weighted_math_average(df):
    weighted_math = df['UGDS'] * df['SATMTMID']
    return int(weighted_math.sum() / df['UGDS'].sum())

In [58]:
college2.groupby('STABBR').apply(weighted_math_average)

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
CO    553
CT    545
DC    621
DE    569
FL    565
GA    540
HI    534
IA    577
ID    509
IL    594
IN    546
KS    491
KY    525
LA    549
MA    597
MD    572
ME    524
MI    586
MN    598
MO    576
MS    527
MT    551
NC    552
ND    546
NE    567
NH    561
NJ    554
NM    529
NV    516
NY    578
OH    569
OK    557
OR    540
PA    553
PR    571
RI    567
SC    549
SD    528
TN    544
TX    548
UT    577
VA    550
VI    390
VT    566
WA    555
WI    593
WV    500
WY    540
dtype: int64

In [59]:
(college2
    .groupby('STABBR')
    .agg(weighted_math_average)
)

KeyError: 'UGDS'

In [63]:
(college2
    .groupby('STABBR')
    ['SATMTMID'] 
    .agg(weighted_math_average)
)

KeyError: 'UGDS'

In [71]:
def weighted_average(df):
    weight_m = df['UGDS'] * df['SATMTMID']
    weight_v = df['UGDS'] * df['SATVRMID']
    wm_avg = weight_m.sum() / df['UGDS'].sum()
    wv_avg = weight_v.sum() / df['UGDS'].sum()
    data = {'w_math_avg': wm_avg,
           'w_verbal_avg': wv_avg,
           'math_avg': df['SATMTMID'].mean(),
           'verbal_avg': df['SATVRMID'].mean(),
           'count': len(df)
    }
    return pd.Series(data)

(college2
    .groupby('STABBR')
    .apply(weighted_average)
    .astype(int)
).head(5)

,w_math_avg,w_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,503,555,503,555,1
AL,536,533,504,508,21
AR,529,504,515,491,16
AZ,569,557,536,538,6
CA,564,539,562,549,72


In [73]:
weighted_average(
    college2.query("STABBR == 'AK'")
)

w_math_avg      503.0
w_verbal_avg    555.0
math_avg        503.0
verbal_avg      555.0
count             1.0
dtype: float64

### How it works...

In [61]:
(college
    .groupby('STABBR')
    .apply(weighted_average)
)

,w_math_avg,w_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,5.548091,6.121651,503.000000,555.000000,10.0
AL,261.895658,260.550109,504.285714,508.476190,96.0
AR,301.054792,287.264872,515.937500,491.875000,86.0
AS,0.000000,0.000000,NaN,NaN,1.0
AZ,61.815821,60.511712,536.666667,538.333333,133.0
CA,165.611830,158.098830,562.902778,549.083333,773.0
CO,270.958708,267.975486,540.214286,537.714286,125.0
CT,254.065075,248.509769,522.500000,517.857143,102.0
DC,473.191221,474.415415,588.333333,589.166667,26.0


### There's more...

In [62]:
from scipy.stats import gmean, hmean
def calculate_means(df):
    df_means = pd.DataFrame(index=['Arithmetic', 'Weighted',
                                   'Geometric', 'Harmonic'])
    cols = ['SATMTMID', 'SATVRMID']
    for col in cols:
        arithmetic = df[col].mean()
        weighted = np.average(df[col], weights=df['UGDS'])
        geometric = gmean(df[col])
        harmonic = hmean(df[col])
        df_means[col] = [arithmetic, weighted,
                         geometric, harmonic]
    df_means['count'] = len(df)
    return df_means.astype(int)
(college2
    .groupby('STABBR')
    .apply(calculate_means)
)

SATMTMID  SATVRMID  count
STABBR                                      
AK     Arithmetic       503       555      1
       Weighted         503       555      1
       Geometric        503       555      1
       Harmonic         503       555      1
AL     Arithmetic       504       508     21
...                     ...       ...    ...
WV     Harmonic         480       472     17
WY     Arithmetic       540       535      1
       Weighted         540       535      1
       Geometric        540       534      1
       Harmonic         540       535      1

[212 rows x 3 columns]

## Grouping by continuous variables

Looking to discover the distribution of airlines for different travel distances. This will allow us to find the airline that makes the m0st flights between 500 - 1,000 miles for example.

Going to use 'cut' function to discretize the distance of each flight flown.

### How to do it...

In [74]:
flights = pd.read_csv('data/flights.csv')
flights

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58487,12,31,4,AA,SFO,DFW,515,5.0,166.0,1464,1045,-19.0,0,0
58488,12,31,4,F9,LAS,SFO,1910,13.0,71.0,414,2050,4.0,0,0
58489,12,31,4,OO,SFO,SBA,1846,-6.0,46.0,262,1956,-5.0,0,0
58490,12,31,4,WN,MSP,ATL,525,39.0,124.0,907,855,34.0,0,0


In [75]:
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]
cuts = pd.cut(flights['DIST'], bins=bins)
cuts

0         (500.0, 1000.0]
1        (1000.0, 2000.0]
2         (500.0, 1000.0]
3        (1000.0, 2000.0]
4        (1000.0, 2000.0]
               ...       
58487    (1000.0, 2000.0]
58488      (200.0, 500.0]
58489      (200.0, 500.0]
58490     (500.0, 1000.0]
58491     (500.0, 1000.0]
Name: DIST, Length: 58492, dtype: category
Categories (5, interval[float64]): [(-inf, 200.0] < (200.0, 500.0] < (500.0, 1000.0] < (1000.0, 2000.0] < (2000.0, inf]]

In [76]:
cuts.value_counts()

(500.0, 1000.0]     20659
(200.0, 500.0]      15874
(1000.0, 2000.0]    14186
(2000.0, inf]        4054
(-inf, 200.0]        3719
Name: DIST, dtype: int64

In [77]:
(flights
    .groupby(cuts)
    ['AIRLINE']
    .value_counts(normalize=True) 
    .round(3)
)

DIST              AIRLINE
(-inf, 200.0]     OO         0.326
                  EV         0.289
                  MQ         0.211
                  DL         0.086
                  AA         0.052
                  UA         0.027
                  WN         0.009
(200.0, 500.0]    WN         0.194
                  DL         0.189
                  OO         0.159
                  EV         0.156
                  MQ         0.100
                  AA         0.071
                  UA         0.062
                  VX         0.028
                  US         0.016
                  NK         0.012
                  B6         0.007
                  F9         0.005
                  AS         0.001
(500.0, 1000.0]   DL         0.206
                  AA         0.144
                  WN         0.138
                  UA         0.131
                  OO         0.106
                  EV         0.101
                  MQ         0.051
                  F9         

### How it works...

### There's more...

In [80]:
(flights
  .groupby(cuts)
  ['AIR_TIME']
  .quantile(q=[.25, .5, .75]) # 25th, 50th and 75th percentile airtime for each distance grouping
  .div(60)
  .round(2)
)

DIST                  
(-inf, 200.0]     0.25    0.43
                  0.50    0.50
                  0.75    0.57
(200.0, 500.0]    0.25    0.77
                  0.50    0.92
                  0.75    1.05
(500.0, 1000.0]   0.25    1.43
                  0.50    1.65
                  0.75    1.92
(1000.0, 2000.0]  0.25    2.50
                  0.50    2.93
                  0.75    3.40
(2000.0, inf]     0.25    4.30
                  0.50    4.70
                  0.75    5.03
Name: AIR_TIME, dtype: float64

In [81]:
labels=['Under an Hour', '1 Hour', '1-2 Hours',
        '2-4 Hours', '4+ Hours']
cuts2 = pd.cut(flights['DIST'], bins=bins, labels=labels)
(flights
   .groupby(cuts2)
   ['AIRLINE']
   .value_counts(normalize=True) 
   .round(3) 
   .unstack() 
)

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
DIST,,,,,,,,,,,,,,
Under an Hour,0.052,NaN,NaN,0.086,0.289,NaN,NaN,0.211,NaN,0.326,0.027,NaN,NaN,0.009
1 Hour,0.071,0.001,0.007,0.189,0.156,0.005,NaN,0.100,0.012,0.159,0.062,0.016,0.028,0.194
1-2 Hours,0.144,0.023,0.003,0.206,0.101,0.038,NaN,0.051,0.030,0.106,0.131,0.025,0.004,0.138
2-4 Hours,0.264,0.016,0.003,0.165,0.016,0.031,NaN,0.003,0.045,0.046,0.199,0.040,0.012,0.160
4+ Hours,0.212,0.012,0.080,0.171,NaN,0.004,0.028,NaN,0.019,NaN,0.289,0.065,0.074,0.046


## Counting the total number of flights between cities

### How to do it...

In [87]:
# total number of flights between an origin and destination airport
flights = pd.read_csv('data/flights.csv')
flights_ct = flights.groupby(['ORG_AIR', 'DEST_AIR']).size()
flights_ct

ORG_AIR  DEST_AIR
ATL      ABE          31
         ABQ          16
         ABY          19
         ACY           6
         AEX          40
                    ... 
SFO      SNA         122
         STL          20
         SUN          10
         TUS          20
         XNA           2
Length: 1130, dtype: int64

In [88]:
flights_ct.loc[[('ATL', 'IAH'), ('IAH', 'ATL')]]

ORG_AIR  DEST_AIR
ATL      IAH         121
IAH      ATL         148
dtype: int64

In [89]:
f_part3 = (flights  # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']] 
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
)
f_part3

,0,1
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DCA,DFW
4,LAX,MCI
...,...,...
58487,DFW,SFO
58488,LAS,SFO
58489,SBA,SFO
58490,ATL,MSP


In [85]:
(flights  # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']] 
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
)

,0,1
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DCA,DFW
4,LAX,MCI
...,...,...
58487,DFW,SFO
58488,LAS,SFO
58489,SBA,SFO
58490,ATL,MSP


In [91]:
# example of above
flights[['ORG_AIR', 'DEST_AIR']].loc[0].sort_values().reset_index(drop=True)

0    LAX
1    SLC
Name: 0, dtype: object

In [90]:
rename_dict = {0:'AIR1', 1:'AIR2'}  
(flights     # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']]
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
  .rename(columns=rename_dict)
  .groupby(['AIR1', 'AIR2'])
  .size()
)

AIR1  AIR2
ABE   ATL      31
      ORD      24
ABI   DFW      74
ABQ   ATL      16
      DEN      46
             ... 
SFO   SNA     122
      STL      20
      SUN      10
      TUS      20
      XNA       2
Length: 1085, dtype: int64

In [91]:
(flights     # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']]
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
  .rename(columns=rename_dict)
  .groupby(['AIR1', 'AIR2'])
  .size()
  .loc[('ATL', 'IAH')]
)

269

In [92]:
(flights     # doctest: +SKIP
  [['ORG_AIR', 'DEST_AIR']]
  .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
  .rename(columns=rename_dict)
  .groupby(['AIR1', 'AIR2'])
  .size()
  .loc[('IAH', 'ATL')]
)

KeyError: ('IAH', 'ATL')

### How it works...

### There's more ...

In [93]:
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
data_sorted[:10]

array([['LAX', 'SLC'],
       ['DEN', 'IAD'],
       ['DFW', 'VPS'],
       ['DCA', 'DFW'],
       ['LAX', 'MCI'],
       ['IAH', 'SAN'],
       ['DFW', 'MSY'],
       ['PHX', 'SFO'],
       ['ORD', 'STL'],
       ['IAH', 'SJC']], dtype=object)

In [94]:
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])
flights_sort2.equals(f_part3.rename(columns={'ORG_AIR':'AIR1',
    'DEST_AIR':'AIR2'}))

False

In [94]:
%%timeit
flights_sort = (flights   # doctest: +SKIP
    [['ORG_AIR', 'DEST_AIR']] 
   .apply(lambda ser:
         ser.sort_values().reset_index(drop=True),
         axis='columns')
)

51.3 s ± 1.07 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [92]:
%%timeit
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sort2 = pd.DataFrame(data_sorted,
    columns=['AIR1', 'AIR2'])

16.2 ms ± 755 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Finding the longest streak of on-time flights (streaks)

### How to do it...

In [95]:
# define series of 0's and 1's
s = pd.Series([0, 1, 1, 0, 1, 1, 1, 0])
s

0    0
1    1
2    1
3    0
4    1
5    1
6    1
7    0
dtype: int64

In [110]:
# cumulate the series
s1 = s.cumsum()
s1

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64

In [97]:
# multiply first series with second series where zeros are the first part of the series
s.mul(s1)

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

In [98]:
# difference between first value above
s.mul(s1).diff()

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

In [99]:
(s
    .mul(s.cumsum()) # s1
    .diff()
    .where(lambda x: x < 0) # if not less than zero ie streak ends then nan
)

0    NaN
1    NaN
2    NaN
3   -2.0
4    NaN
5    NaN
6    NaN
7   -5.0
dtype: float64

In [100]:
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
    .ffill() # fill downwards
)

0    NaN
1    NaN
2    NaN
3   -2.0
4   -2.0
5   -2.0
6   -2.0
7   -5.0
dtype: float64

In [101]:
(s
    .mul(s.cumsum())
    .diff()
    .where(lambda x: x < 0)
    .ffill()
    .add(s.cumsum(), fill_value=0) # s1 # fill nan's with zero
)

0    0.0
1    1.0
2    2.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64

In [102]:
flights = pd.read_csv('data/flights.csv')
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    [['AIRLINE', 'ORG_AIR', 'ON_TIME']]
)

,AIRLINE,ORG_AIR,ON_TIME
0,WN,LAX,0
1,UA,DEN,1
2,MQ,DFW,0
3,AA,DFW,1
4,WN,LAX,0
...,...,...,...
58487,AA,SFO,1
58488,F9,LAS,1
58489,OO,SFO,1
58490,WN,MSP,0


In [103]:
def max_streak(s):
    s1 = s.cumsum()
    return (s
       .mul(s1)
       .diff()
       .where(lambda x: x < 0) 
       .ffill()
       .add(s1, fill_value=0)
       .max()
    )

In [104]:
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    .sort_values(['MONTH', 'DAY', 'SCHED_DEP']) 
    .groupby(['AIRLINE', 'ORG_AIR'])
    ['ON_TIME'] 
    .agg(['mean', 'size', max_streak])
    .round(2)
)

mean  size  max_streak
AIRLINE ORG_AIR                        
AA      ATL      0.82   233          15
        DEN      0.74   219          17
        DFW      0.78  4006          64
        IAH      0.80   196          24
        LAS      0.79   374          29
...               ...   ...         ...
WN      LAS      0.77  2031          39
        LAX      0.70  1135          23
        MSP      0.84   237          32
        PHX      0.77  1724          33
        SFO      0.76   445          17

[114 rows x 3 columns]

### How it works...

### There's more... (come back)

In [105]:
def max_delay_streak(df):
    df = df.reset_index(drop=True)
    late = 1 - df['ON_TIME']
    late_sum = late.cumsum()
    streak = (late
        .mul(late_sum)
        .diff()
        .where(lambda x: x < 0) 
        .ffill()
        .add(late_sum, fill_value=0)
    )
    last_idx = streak.idxmax()
    first_idx = last_idx - streak.max() + 1
    res = (df
        .loc[[first_idx, last_idx], ['MONTH', 'DAY']]
        .assign(streak=streak.max())
    )
    res.index = ['first', 'last']
    return res

In [109]:
(flights
    .assign(ON_TIME=flights['ARR_DELAY'].lt(15).astype(int))
    .sort_values(['MONTH', 'DAY', 'SCHED_DEP']) 
    .groupby(['AIRLINE', 'ORG_AIR']) 
    .apply(max_delay_streak) 
#     .sort_values('streak', ascending=False)
)

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'